#<center>**SỬ DỤNG MÔ HÌNH XXX VÀ YYY TÌM KIẾM VĂN BẢN TIẾNG ANH**
#<center>**Môn học:** Truy xuất thông tin
#<center>**Mã lớp:** CS419.N11
---
**Danh sách sinh viên thực hiện:**
1.  Phan Thanh Hải - 18520705
2. Nguyễn Hoàng Long - 20520239

**Lưu ý:**

*   Quy cách đặt tên các thành phần cơ bản của chương trình theo hướng dẫn sau đây: https://peps.python.org/pep-0008/.
*   Sử dụng tiếng Anh để đặt tên biến, tên hàm, v.v... Chỉ sử dụng tiếng Việt để comment, nếu cần.

# **1. Chèn các thư viện cần thiết**

In [29]:
import os
import re
import nltk
nltk.download('all')
from natsort import natsorted
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Đọc dữ liệu**

In [ ]:
'''
Input: chuỗi kí tự documents_path là đường dẫn tới thư mục chứa các tài liệu
Output: danh sách documents, trong đó mỗi phần tử của danh sách là toàn bộ chuỗi kí tự của tập tài liệu tương ứng
Ví dụ: documents = ['the specific configuration . the experiment . ',
                    'the discussion here is restricted . two dimensional incompressible steady flow .']
'''
def read_documents(documents_path):
    documents = []
    os.chdir(documents_path)
    # Use natsorted() for natural sorting of file names
    for file in natsorted(os.listdir()):
    # Check whether file is in text format or not
        if file.endswith(".txt"):
            file_path = f"{documents_path}/{file}"
            with open(file_path, 'r') as f:
                documents.append(f.read())    
    return documents

In [ ]:
documents_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/Cranfield'
documents = read_documents(documents_path)
print(documents[:5])

['experimental investigation of the aerodynamics of a wing in a slipstream . an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem . the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary layer control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory . an empirical evaluation of the destalling effects was made for the specific configuration of the experiment . ', "simple shear flow past a flat plate in an incompressible fluid of small viscosity . in th

In [ ]:
'''
Input: chuỗi kí tự queries_path là đường dẫn tới tập tin chứa các câu truy vấn
Output: danh sách queries, trong đó mỗi phần tử của danh sách là toàn bộ chuỗi kí tự của câu truy vấn tương ứng
Ví dụ: queries = ['aeroelastic models of heated high speed aircraft .',
                  'with flight of high speed aircraft .']
'''
def read_queries(queries_path):
    with open(queries_path) as file:
        queries = file.readlines()
    # Mỗi dòng trong tập tin query.txt có cấu trúc [id]\t[câu truy vấn]
    # -> Ta lấy câu truy vấn bằng cách tách từng dòng bởi dấu \t và chỉ lấy phần sau sau dấu \t
    for idx in range(len(queries)):
        queries[idx] = queries[idx][:-1].split("\t")[1]
    return queries

In [ ]:
queries_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/TEST/query.txt'
queries = read_queries(queries_path)
print(queries[:5])

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .', 'what are the structural and aeroelastic problems associated with flight of high speed aircraft .', 'what problems of heat conduction in composite slabs have been solved so far .', 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium .', 'what chemical kinetic system is applicable to hypersonic aerodynamic problems .']


In [60]:
len(queries)

225

In [ ]:
'''
Input: chuỗi kí tự relevance_path là đường dẫn tới thư mục chứa kết quả truy vấn đúng
Output: từ điển relevance, trong đó:
+ thành phần khóa là id của câu truy vấn
+ thành phần value là danh sách id của tập tài liệu tương ứng với kết quả truy vấn đúng dựa trên id của câu truy vấn trên
Ví dụ: relevance = {1: [1, 2, 3]
                    2: [3, 4]}
'''
def read_relevance(relevance_path):
    relevance = {}
    os.chdir(relevance_path)
    idx = 1
    # Use natsorted() for natural sorting of file names
    for file in natsorted(os.listdir()):
    # Check whether file is in text format or not
        if file.endswith(".txt"):
            file_path = f"{relevance_path}/{file}"
            with open(file_path, 'r') as f:
                line = f.readlines()
                relevance[idx] = []
                for li in line:
                    temp = re.split(r"[ \t\n]\s*", li)
                    relevance[idx].append(int(temp[1]))
        idx += 1
    return relevance

In [ ]:
relevance_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/TEST/RES'
relevance = read_relevance(relevance_path)
print(relevance)

{1: [184, 29, 31, 12, 51, 102, 13, 14, 15, 57, 378, 859, 185, 30, 37, 52, 142, 195, 875, 56, 66, 95, 462, 497, 858, 876, 879, 880, 486], 2: [12, 15, 184, 858, 51, 102, 202, 14, 52, 380, 746, 859, 948, 285, 390, 391, 442, 497, 643, 856, 857, 877, 864, 658, 486], 3: [5, 6, 90, 91, 119, 144, 181, 399, 485], 4: [236, 166, 488], 5: [552, 401, 1297, 1296, 488], 6: [99, 115, 257, 258, 491], 7: [20, 56, 57, 58, 19, 492], 8: [48, 122, 20, 58, 196, 354, 360, 197, 999, 1112, 1005, 492], 9: [21, 22, 550, 534], 10: [259, 405, 302, 436, 437, 438, 998, 1011, 493], 11: [27, 28, 262, 160, 20, 263, 654, 495], 12: [86, 194, 650, 649, 652, 624], 13: [64, 265, 65, 311, 496], 14: [64, 65, 496], 15: [463, 462, 497], 16: [266, 106, 196, 498], 17: [106, 196, 498], 18: [196, 197, 198, 498], 19: [32, 67, 164, 639, 715, 716, 719, 1379, 717, 499], 20: [87, 88, 104, 267, 268, 269, 270, 407, 408, 500], 21: [271, 16, 413, 414, 502], 22: [68, 502], 23: [900, 902, 200, 201, 601, 899, 903, 593, 199, 594, 901, 544, 597, 

# Sử dụng whoosh

In [ ]:
pip install whoosh flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ind

In [39]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.analysis import StandardAnalyzer
from whoosh.writing import AsyncWriter

schema = Schema(docid=STORED(), content=TEXT(stored=True, analyzer=StandardAnalyzer()))
ix = create_in("ind", schema)
writer = AsyncWriter(ix)
for i in range(len(documents)):
  writer.add_document(docid="{}".format(i+1), content=documents[i])
writer.commit()

In [36]:
queries[0]

'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .'

In [106]:
from whoosh import qparser
from whoosh import scoring
import whoosh.index as index

ind = index.open_dir("ind")
searcher = ind.searcher(weighting=scoring.TF_IDF())

In [107]:
def use_whoose(query):
    parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)
    query = parser.parse(queries[0])
    results = searcher.search(query, limit=None)
    ans = []
    for i in range(len(results)):
        ans.append(int(results[i]["docid"]))
    return ans

In [102]:
xyz = use_whoose(queries[0])
xyz1 = set(xyz[:14])
xyz2 = set(relevance[1])
xyz3= len(xyz1.intersection(xyz2))
print('Precision:', xyz3/len(xyz1))
print('Recall:', xyz3/len(xyz2))

Precision: 0.42857142857142855
Recall: 0.20689655172413793


In [112]:
prec_lst = []
recall_lst = []
for i in range(225):
    xyz = use_whoose(queries[i])
    xyz1 = set(xyz)
    xyz2 = set(relevance[i + 1])
    xyz3= len(xyz1.intersection(xyz2))
    prec_lst.append(xyz3/len(xyz1))
    recall_lst.append(xyz3/len(xyz2))

In [111]:
prec = sum(prec_lst) / len(prec_lst)
rec = sum(recall_lst) / len(recall_lst)
print('Precision:', prec)
print('Recall:', rec)
print((2*prec*rec)/(prec+rec))

Precision: 0.006332665330661314
Recall: 0.3723777399074151
0.012453545351832917


In [109]:
print(prec_lst)
error_idx = []
for i in range(len(prec_lst)):
  if prec_lst[i] == 0:
      error_idx.append(i)

[0.04408817635270541, 0.04008016032064128, 0.0, 0.002004008016032064, 0.006012024048096192, 0.0, 0.01002004008016032, 0.012024048096192385, 0.0, 0.008016032064128256, 0.01002004008016032, 0.002004008016032064, 0.002004008016032064, 0.0, 0.002004008016032064, 0.002004008016032064, 0.0, 0.004008016032064128, 0.012024048096192385, 0.004008016032064128, 0.004008016032064128, 0.002004008016032064, 0.02404809619238477, 0.006012024048096192, 0.012024048096192385, 0.004008016032064128, 0.0, 0.0, 0.006012024048096192, 0.006012024048096192, 0.004008016032064128, 0.0, 0.006012024048096192, 0.008016032064128256, 0.0, 0.002004008016032064, 0.006012024048096192, 0.01002004008016032, 0.012024048096192385, 0.014028056112224449, 0.004008016032064128, 0.002004008016032064, 0.002004008016032064, 0.002004008016032064, 0.012024048096192385, 0.01603206412825651, 0.012024048096192385, 0.01603206412825651, 0.0, 0.002004008016032064, 0.004008016032064128, 0.0, 0.006012024048096192, 0.01002004008016032, 0.00601

In [88]:
for idx in error_idx:
    print(queries[idx])

what problems of heat conduction in composite slabs have been solved so far .
what theoretical and experimental guides do we have as to turbulent couette flow behaviour .
papers on internal /slip flow/ heat transfer studies .
papers on shock-sound wave interaction .
can the three-dimensional problem of a transverse potential flow about a body of revolution be reduced to a two-dimensional problem .
how is the design of ring or part ring wings by linear theory affected by thickness .
what application has the linear theory design of curved wings .
to find an approximate correction for thickness in slender thin-wing theory .
are there any papers dealing with acoustic wave propagation in reacting gases .
can the three-point boundary-value problem for the blasius equation be integrated numerically,  using suitable transformations,  without iteration on the boundary conditions .
what is the available information pertaining to the effect of slight rarefaction on boundary layer flows (the slip 

# **3. Thống kê mục từ**

Ta sẽ xây dựng cấu trúc dữ liệu để biểu diễn thông tin mục từ của tài liệu.
*word_lst*: từ điển lưu trữ thông tin của các mục từ bao gồm nội dung chuỗi kí tự, chỉ số tài liệu và tần số xuất hiện trong tài liệu tương ứng theo cấu trúc sau:

```
word_lst = {
[mục từ 1]: { [chỉ số tài liệu 1]: [tần số],
	          [chỉ số tài liệu 2]: [tần số],
	        … }
[mục từ 2]: { [chỉ số tài liệu 1]: [tần số],
	          [chỉ số tài liệu 2]: [tần số],
	        … }
… }
```




In [ ]:
'''
Hàm này dùng để cập nhật thông tin của các mục từ trong từ điển word_lst
# Input:
# + id: id của câu
# + term: term của tài liệu
# + word_lst: từ điển word_lst chứa thông tin
# Output: từ điển word_lst sau khi đã khi đã cập nhật thông tin term
'''
def update_word_lst(id, word, word_lst):
    if word in word_lst:
        if id in word_lst[word]:
            # Cập nhật tần số xuất hiện của từ đó lên 1
            word_lst[word][id] += 1
        else:
            # Thêm chỉ số tài liệu và tần số = 1 tương ứng với mục từ mới
            word_lst[word][id] = 1
    else:
        # Thêm mục từ mới vào trong word_lst và chỉ số tài liệu, tần số = 1 tương ứng
        word_lst[word] = {id: 1}
    return word_lst

In [ ]:
# Hàm này dùng để phân tách mục từ từ 1 câu
def create_words(sentence):
    words = [] 
    # Remove punctuation and numbers. Split on whitespace
    regex = r"[ .,()0123456789=:+-/']\s*"
    words = re.split(regex, sentence)
    # Use filter(None, ) to remove empty element after splitting string
    words = filter(None, words)
    return words

In [ ]:
def create_vocab_lst(inverted_index):
    vocab_lst = {}
    for term in inverted_index:
        freq = len(inverted_index[term])
        no_of_docs = sum(inverted_index[term].values())
        vocab_lst[term] = [freq, no_of_docs]
    return vocab_lst

In [ ]:
# Hàm này dùng để cập nhật thông tin toàn bộ mục từ của 1 câu trong từ điển word_lst
def create_word_lst(id, sentence, word_lst):
    words = create_words(sentence)
    for word in words:
        # Update word in word_lst
        word_lst = update_word_lst(id, word, word_lst)
    return word_lst

In [ ]:
word_lst = {}
idx = 1
for document in documents:
    word_lst = create_word_lst(idx, document, word_lst)
    idx += 1
print(word_lst)

In ra màn hình kết quả thống kê các mục từ có trong tập tài liệu, trong đó mỗi dòng có định dạng: 

```
<mục từ> <số lượng tài liệu chứa mục từ> <tần số>
```



In [ ]:
word_info_lst = create_vocab_lst(word_lst)
res_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/word_lst.txt'
with open(res_path, 'w') as file:
    for word in word_info_lst:
        string = word + ' ' + str(word_info_lst[word][0]) + ' ' + str(word_info_lst[word][1]) + '\n'
        file.write(string)

In [ ]:
print('Số lượng mục từ có trong toàn bộ tập tài liệu là:', len(word_info_lst))

# **4. Xác định term**

*Terms* của tài liệu được xác định theo các bước như sau:

**Bước 1.** Ngắt các token trong tài liệu bởi dấu cách.

**Bước 2.** Với mỗi token:
*   **Bước 2.1.** Loại bỏ các kí tự dấu câu và các chữ số.
*   **Bước 2.2.** Loại bỏ các token là *stopwords* trong tài liệu.
*   **Bước 2.3.** Sử dụng kĩ thuật *stemming* để lấy từ gốc trong câu.
*   **Bước 2.4.** Loại bỏ những từ gốc chỉ có chứa 3 kí tự.

Kết quả sau **bước 2** cho ta danh sách các *terms* của tài liệu.

Ta sẽ xây dựng cấu trúc dữ liệu để biểu diễn thông tin mục từ của tài liệu.
*inverted_indext*: từ điển lưu trữ thông tin của các *terms* bao gồm nội dung chuỗi kí tự, chỉ số tài liệu và tần số xuất hiện trong tài liệu tương ứng theo cấu trúc sau:

```
inverted_index = {
[term 1]: { [chỉ số tài liệu 1]: [tần số],
	        [chỉ số tài liệu 2]: [tần số],
	       … }
[term 2]: { [chỉ số tài liệu 1]: [tần số],
	        [chỉ số tài liệu 2]: [tần số],
	      … }
… }
```

In [ ]:
'''
Hàm này dùng để cập nhật thông tin của các mục từ trong từ điển inverted_file
# Input:
# + id: id của câu
# + term: term của tài liệu
# + inverted_index: từ điển inverted_index chứa thông tin
# Output: từ điển inverted_index sau khi đã khi đã cập nhật thông tin term
'''
def update_inverted_index(id, term, inverted_index):
    if term in inverted_index:
        if id in inverted_index[term]:
            inverted_index[term][id] += 1
        else:
            inverted_index[term][id] = 1
    else:
        inverted_index[term] = {id: 1}
    return inverted_index

In [ ]:
# Hàm này dùng để phân tách term từ 1 câu
def create_terms(sentence):
    terms = []
    # Remove punctuation and numbers. Split on whitespace
    regex = r"[ .,()0123456789=:+-/']\s*"
    word_lst = re.split(regex, sentence)
    # Use filter(None, ) to remove empty element after splitting string
    word_lst = filter(None, word_lst)
    for word in word_lst:
        # Remove stopwords
        stopword_lst = stopwords.words('english')
        if word in stopword_lst:
            continue
        # Perform stemming
        stemmer = SnowballStemmer("english")
        stem_word = stemmer.stem(word)
        # Remove 1 character
        if len(stem_word) < 3:
            continue
        terms.append(stem_word)
    return terms

In [ ]:
# Hàm này dùng để cập nhật thông tin toàn bộ terms của 1 câu trong từ điển inverted_index
# Input:
# + id: id của câu
# + term: term của tài liệu
# + inverted_index: từ điển inverted_index chứa thông tin
# Output: từ điển inverted_index sau khi đã khi đã cập nhật thông tin toàn bộ terms
def create_inverted_index(id, sentence, inverted_index):
    terms = create_terms(sentence)
    for term in terms:
        # Update term in inverted_index
        inverted_index = update_inverted_index(id, term, inverted_index)
    return inverted_index

In [ ]:
inverted_index = {}
idx = 1
for document in documents:
    inverted_index = create_inverted_index(idx, document, inverted_index)
    idx += 1
print(inverted_index)

# **5. Lập chỉ mục**

In ra màn hình danh sách tập từ vựng, trong đó mỗi dòng có định dạng: 

```
<term> <số lượng tài liệu chứa term> <tần số>
```



In [ ]:
vocab_lst = create_vocab_lst(inverted_index)
res_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/vocab_lst.txt'
with open(res_path, 'w') as file:
    for term in vocab_lst:
        string = term + ' ' + str(vocab_lst[term][0]) + ' ' + str(vocab_lst[term][1]) + '\n'
        file.write(string)

In [ ]:
print('Số lượng terms có trong toàn bộ tập tài liệu là:', len(inverted_index))

In ra màn hình danh sách posting, trong đó mỗi dòng có định dạng: 

```
<term> <chỉ số tài liệu> <tần số>
```


In [ ]:
res_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/posting_lst.txt'
with open(res_path, 'w') as file:
    for term in inverted_index:
        idx = 1
        for id in inverted_index[term]:
            string = term + ' ' + str(id) + ' ' + str(inverted_index[term][id]) + '\n'
            file.write(string)

# **6. Tìm kiếm theo mô hình boolean**

In [ ]:
'''
Input:
+ query: câu truy vấn
+ inverted_index: từ điển chứa thông tin
Output: danh sách index_lst chứa danh sách id của tài liệu tương ứng với
câu truy vấn query sử dụng mô hình tìm kiếm boolean chỉ sử dụng phép toán OR
'''
def boolean_retrieval(query, inverted_index):
    retrieved_indexes = []
    query_terms = create_terms(query)
    for term in query_terms:
        if term in inverted_index:
            retrieved_indexes += inverted_index[term]
    return list(set(retrieved_indexes))

In [ ]:
print(boolean_retrieval(queries[1], inverted_index))

## **6.2 Tìm kiếm theo mô hình Vector Space**

In [ ]:
def termFrequencyInDoc(word_info_lst, documents):
    tf_docs = {}
    for term in word_info_lst.keys():
        tf_docs[term] = {}
        for docs_id in range(len(documents)):
            doc = documents[docs_id]
            tf_docs[term][docs_id] = np.log2(1 + doc.count(term))
    return tf_docs
tf_score = termFrequencyInDoc(word_info_lst, documents)

In [ ]:
def inverseFrequencyInDoc(word_info_lst, documents):
    idf_docs = {}
    length = len(documents)
    for term in word_info_lst.keys():
        a = word_info_lst[term][0]
        idf_docs[term] = np.log2(length / (1 + a))
    return idf_docs
idf_score = inverseFrequencyInDoc(word_info_lst, documents)

In [ ]:
def tf_idf_score(tf_weight, idf_weight, word_info_lst, documents):
    tf_idf_score = {}
    for term in word_info_lst.keys():
        tf_idf_score[term] = {}
        for docs_id in range(len(documents)):
            doc = documents[docs_id]
            tf_idf_score[term][docs_id] = tf_weight[term][docs_id] * idf_weight[term]
    return tf_idf_score
tf_idf = tf_idf_score(tf_score, idf_score, word_info_lst, documents)


In [ ]:
def vectorSpaceModel(query_input, documents, tf_idf, word_info_lst, number_of_selected):
    query_vocab = []
    query = list(create_words(query_input))
    for word in query:
        flag = False
        for term in word_info_lst:
            if word == term: flag = True
        if flag == True:
            if word not in query_vocab:
                query_vocab.append(word)
    query_wc = {}
    for word in query_vocab:
        query_wc[word] = query.count(word)
        
    relevance_scores = {}
    for docs_id in range(len(documents)):
        score = 0
        for word in query_vocab:
            score += query_wc[word] * tf_idf[word][docs_id]
        relevance_scores[docs_id + 1] = score
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key=lambda x: x[1], reverse = True))
    top_values = {k: sorted_value[k] for k in list(sorted_value)[:number_of_selected]}
    return top_values

## **6.3 Tìm kiếm với mô hình Latent Semantic Index (LSI)**

In [ ]:
def LSI(queries, documents, number_of_selected):
    query = [queries]
    D = documents
    # Vector hoá tập tài liệu có dạng (chỉ số tài liệu, keys của term) = tần số của term trong tài liệu đó
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(D)
    A = X.T.toarray()
    # Phân tích ma trận tài liệu thành ba ma trận theo phương pháp Singular Value Decomposition (SVD) 
    # Sử dụng thư viện numpy.linalg.svd để phân tích
    [u,s,vt] = svd(A, full_matrices=False)

    # Vector hoá câu truy vấn 
    q = vectorizer.transform(query)
    q = q.toarray()
    
    q = np.matmul(np.matmul(q,u),np.linalg.inv(np.diag(s)))
    result = []
    for i in range(len(vt)):
        value = np.dot(q,vt[:,i])/(np.linalg.norm(q)*np.linalg.norm(vt[:,i]))
        result.append(value)
    result_sorted = sorted(result)[-number_of_selected:]
    
    # Lấy ra những tài liệu có độ chính xác cao nhất khi so khớp câu truy vấn
    top_docs = []
    for value in result_sorted:
        top_docs.append(result.index(value) + 1)
    return top_docs

# **7. Duyệt sơ lược kết quả và nhận xét**

In [ ]:
# Dùng for duyệt một số câu truy vấn đầu. So sánh đối chiếu với
# kết quả trong relevance và đưa ra nhận xét
res = []
precision_lst = []
recall_lst = []
f_score_lst = []
no_queries = len(queries)
for idx in range(0, no_queries):
    retrieved_indexes = set(boolean_retrieval(queries[idx], inverted_index))
    relevant_indexes = set(relevance[idx+1])
    no_relevant_docs_retrieved = len(retrieved_indexes & relevant_indexes)
    
    precision, recall, f_score = 0, 0, 0

    if len(retrieved_indexes) != 0:
        precision = round(no_relevant_docs_retrieved/len(retrieved_indexes), 2)
    if len(relevant_indexes) != 0:
        recall = round(no_relevant_docs_retrieved/len(relevant_indexes), 2)
    if precision * recall != 0:
        f_score = round((2 * precision * recall) / (precision + recall), 2)
    
    precision_lst.append(precision)
    recall_lst.append(recall)
    f_score_lst.append(f_score)

In ra màn hình kết quả độ đo sử dụng đánh giá mô hình bài toán, trong đó mỗi dòng có định dạng: 

```
<chỉ số của câu truy vấn> <Độ chính xác> <Độ phủ> <Độ đo F>
```



In [ ]:
res_path = '/content/drive/MyDrive/FOLDER MÔN HỌC/HK1(2022-2023)/CS419.N11 - Truy xuất thông tin/Bài tập/experimental_result.txt'
with open(res_path, 'w') as file:
    for idx in range(len(precision_lst)):
        string = str(idx+1) + ' ' + str(precision_lst[idx]) + ' ' + str(recall_lst[idx]) + ' ' + str(f_score_lst[idx]) + '\n'
        file.write(string)

In [ ]:
print('Độ chính xác trung bình:', round(sum(precision_lst)/len(precision_lst), 2))
print('Độ phủ trung bình:', round(sum(recall_lst)/len(recall_lst), 2))
print('Độ đo F trung bình:', round(sum(f_score_lst)/len(f_score_lst), 2))

## **7.2 Đánh giá độ chính xác của mô hình Vector Space**

In [ ]:
'''
Dùng mô hình truy xuất thông tin Vector Space, 
output trả về là toàn bộ tài liệu.
'''
avg_precision_VSM = []
avg_recall_VSM = []
avg_f_score_VSM = []
precision_lst_VSM = []
recall_lst_VSM = []
f_score_lst_VSM = []
beta = 1.5
no_queries = len(queries)

# list_num_docs = [5,10,20,30,40,50,60,80,100,150,225]   
list_num_docs = [15]
for num_docs in list_num_docs:
    for query_index in range(0, no_queries):
        retrieved_indexes = set(vectorSpaceModel(queries[query_index], documents, tf_idf, word_info_lst, num_docs))
        relevant_indexes = set(relevance[query_index + 1])
        # Lập danh sách số lượng tài liệu phù hợp nhất 
        # retrieved_indexes_set = set(retrieved_indexes[:_step_docs])
        no_relevant_docs_retrieved = len(retrieved_indexes & relevant_indexes)
        precision, recall, f_score, e_measure = 0, 0, 0, 0
        if len(retrieved_indexes) != 0:
            precision = round(no_relevant_docs_retrieved/len(retrieved_indexes), 2)
        if len(retrieved_indexes) != 0:
            recall = round(no_relevant_docs_retrieved/len(relevant_indexes), 2)
        if precision * recall != 0:
            f_score = round((2 * precision * recall) / (precision + recall), 2)
        precision_lst_VSM.append(precision)
        recall_lst_VSM.append(recall)
        f_score_lst_VSM.append(f_score)
    avg_precision_VSM.append(round(sum(precision_lst_VSM) / len(precision_lst_VSM), 2))
    avg_recall_VSM.append(round(sum(recall_lst_VSM) / len(recall_lst_VSM), 2))
    avg_f_score_VSM.append(round(sum(f_score_lst_VSM)/len(f_score_lst_VSM), 2))

In [ ]:
arr = np.column_stack((precision_lst_VSM, recall_lst_VSM))
import pandas as pd
evaluation = arr[arr[:, 1].argsort()]
recall_sorted = []
precision_sorted = []
for idx in range(evaluation.shape[0]):
    precision_sorted.append(evaluation[idx][0])
    recall_sorted.append(round(evaluation[idx][1],1))
df = pd.DataFrame({"Recall": recall_sorted, "Precision": precision_sorted})
df_groupby = df.groupby(["Recall"]).mean()
df_groupby

In [ ]:
from sklearn.metrics import auc, precision_recall_curve
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6,6))
plt.plot(df_groupby, 'r')
plt.title("Precision - Recall Curve of Vector Space Model with top 15 docs")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")

In [ ]:
print('Độ chính xác trung bình :', round(sum(precision_lst_VSM)/len(precision_lst_VSM), 2))
print('Độ phủ trung bình:', round(sum(recall_lst_VSM)/len(recall_lst_VSM), 2))
print('Độ đo F trung bình:', round(sum(f_score_lst_VSM)/len(f_score_lst_VSM), 2))


## 7.3 Đánh giá độ chính xác của mô hình Latent Semantic Index

In [ ]:
'''
Dùng mô hình truy xuất thông tin LSI, 
output trả về là toàn bộ tài liệu.
'''
avg_precision_LSI = []
avg_recall_LSI = []
precision_lst_LSI = []
recall_lst_LSI = []
f_score_lst_LSI = []
avg_f_score_LST = []
no_queries = len(queries)

# list_num_docs = [5,10,20,30,40,50,60,80,100,150,225]   
list_num_docs = [15]
for num_docs in list_num_docs:
    for query_index in range(0, no_queries):
        retrieved_indexes = set(LSI(queries[query_index], documents, num_docs))
        relevant_indexes = set(relevance[query_index + 1])
        # Lập danh sách số lượng tài liệu phù hợp nhất 
        # retrieved_indexes_set = set(retrieved_indexes[:_step_docs])
        no_relevant_docs_retrieved = len(retrieved_indexes & relevant_indexes)
        precision, recall, f_score = 0, 0, 0
        if len(retrieved_indexes) != 0:
            precision = round(no_relevant_docs_retrieved/len(retrieved_indexes), 2)
        if len(retrieved_indexes) != 0:
            recall = round(no_relevant_docs_retrieved/len(relevant_indexes), 2)
        if precision * recall != 0:
            f_score = round((2 * precision * recall) / (precision + recall), 2)
        precision_lst_LSI.append(precision)
        recall_lst_LSI.append(recall)
        f_score_lst_LSI.append(f_score)
    avg_precision_LSI.append(round(sum(precision_lst_LSI) / len(precision_lst_LSI), 2))
    avg_recall_LSI.append(round(sum(recall_lst_LSI) / len(recall_lst_LSI), 2))
    avg_f_score_LST.append(round(sum(f_score_lst_LSI)/len(f_score_lst_LSI), 2))

In [ ]:
arr = np.column_stack((precision_lst_LSI, recall_lst_LSI))
import pandas as pd
evaluation = arr[arr[:, 1].argsort()]
recall_sorted = []
precision_sorted = []
for idx in range(evaluation.shape[0]):
    precision_sorted.append(evaluation[idx][0])
    recall_sorted.append(round(evaluation[idx][1],1))
df = pd.DataFrame({"Recall": recall_sorted, "Precision": precision_sorted})
df_groupby = df.groupby("Recall").mean()
df_groupby["Precision"]

In [ ]:
print('Độ chính xác trung bình:', round(sum(precision_lst_LSI)/len(precision_lst_LSI), 2))
print('Độ phủ trung bình:', round(sum(recall_lst_LSI)/len(recall_lst_LSI), 2))
print('Độ đo F trung bình:', round(sum(f_score_lst_LSI)/len(f_score_lst_LSI), 2))

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.plot(df_groupby, 'r')
plt.title("Precision - Recall Curve of LSI Model top 15 docs")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")